#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2017


# Homework 2:  Link Analysis -- PageRank + SEO

### 100 points [5% of your final grade]

### Due: Tuesday, February 21, 2017 by 11:59pm

*Goals of this homework:* Explore real-world challenges of building a graph (in this case, from tweets), implement and test PageRank over this graph, and investigate factors that impact a page's rank on Google and Bing.

*Submission Instructions:* To submit your homework, rename this notebook as YOUR_UIN_hw2.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Friday, February 24, 2017 at 11:59pm.

# Part 1: PageRank (70 points)

## A Twitter-Mentioning Graph

In this assignment, we're going to adapt the classic PageRank approach to allow us to find not the most authoritative web pages, but rather to find significant Twitter users. So, instead of viewing the world as web pages with hyperlinks (where pages = nodes, hyperlinks = edges), we're going to construct a graph of Twitter users and their mentions of other Twitter users (so user = node, mention of another user = edge). Over this Twitter-user graph, we can apply the PageRank approach to order the users. The main idea is that a user who is mentioned by other users is more "impactful". 

Here is a toy example. Suppose you are given the following four tweets:

* **userID**: diane, **text**: "@bob Howdy!"
* **userID**: charlie, **text**: "Welcome @bob and @alice!"
* **userID**: bob, **text**: "Hi @charlie and @alice!"
* **userID**: alice, **text**: "Howdy!"

There are four short tweets generated by four users. The @mentions between users form a directed graph with four nodes and five edges. E.g., the "diane" node has a directed edge to the "bob" node. Note that a retweet also contain the "@", so it should be counted as a mention as well.

You should build a graph by parsing the tweets in the file we provide called *pagerank.json*.

**Notes:**

* The edges are binary and directed. If Bob mentions Alice once, in 10 tweets, or 10 times in one tweet, there is an edge from Bob to Alice, but there is not an edge from Alice to Bob.
* If a user mentions herself, ignore it.
* Correctly parsing @mentions in a tweet is error-prone. Use the entities field.
* Later you will need to implement the PageRank algorithm on the graph you build here.


In [1]:
# Insert as many cells as you want

import json
import pprint
import sys
import time
import numpy as np
import math
from __future__ import division
import operator

In [2]:
# Here define your function for building the graph by parsing the input file of tweets

def build_graph(your_json):
    
    users = {} #(key,value) - (user-id, index)
    users_id_names = {} #(key,value) - (user-id, user-name)
    graph = {} #(key,value) - (user-index, list of user-indices it mentions)

    with open(your_json) as f:
        for line in f:
            temp = json.loads(line) #every tweet (a line in json)
            temp_user = temp['user']['id'] #user-id of every tweet

            #adding a user which the parser encounters first time to various dictionaries
            if temp_user not in users:
                users[temp_user] = len(users) + 1  #index of user based on when it was first seen in data
                users_id_names[temp_user] = temp['user']['screen_name'] #dict for screen names and associated ids
                if len(temp['entities']['user_mentions']) > 0: graph[users[temp_user]] = [] #instantiating a user's mentions list


            for item in temp['entities']['user_mentions']: #iterating over all mentions of a tweet
                #adding users already not there, repeating above code block
                if item['id'] not in users:
                    users[item['id']] = len(users) + 1
                    users_id_names[item['id']] = item['screen_name']
                    graph[users[item['id']]] = []

                #then checking if a user mention isn't already in current tweet user's mention list, if not add.
                if (users[item['id']] not in graph[users[temp_user]]) & (item['id'] != temp_user):
                    graph[users[temp_user]].append(users[item['id']])


    return users, users_id_names, graph

In [3]:
#Call the function to generate graph with data
#yourGraph is a dictionary where (k,v) = (node,connect-nodes)
start = time.time()
yourUsers, yourUsersNames, yourGraph = build_graph('pagerank.json')
yourUsers1, yourUsersNames1, yourGraph1 = build_graph('try.json')
end = time.time()

print "Time taken to build graph is %04.2f seconds"%(end - start)

Time taken to build graph is 0.58 seconds


In [4]:
#Access username of certain user in the graph nodes. Just there to check for a certain user.
print yourUsersNames[yourUsers.keys()[yourUsers.values().index(15)]]

twerkitnarry


In [5]:
# Call your function to print out the size of the graph, i.e., the number of nodes and edges
# How you maintain the graph is totaly up to you
#checking characteristics of final graph
nodes = 0
edges = 0
for item in yourGraph:
    nodes+= 1
    edges += len(yourGraph[item])

print nodes, edges
#print sys.getsizeof(yourGraph)

16430 24256


We will not check the correctness of your graph. However, this will affect the PageRank results later.

## PageRank Implementation

Your program will return the top 10 users with highest PageRank scores. The **output** should be like:

* user1 - score1
* user2 - score2
* ...
* user10 - score10

You should follow these **rules**:

* Assume all nodes start out with equal probability.
* The probability of the random surfer teleporting is 0.1 (that is, the damping factor is 0.9).
* If a user is never mentioned and does not mention anyone, their PageRank scores should be zero. Do not include the user in the calculation.
* It is up to you to decide when to terminate the PageRank calculation.
* There are PageRank implementations out there on the web. Remember, your code should be **your own**.


**Hints**:
* If you're using the matrix style approach, you should use [numpy.matrix](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matrix.html).
* Scipy is built on top of Numpy and has support for sparse matrices. You most likely will not need to use Scipy unless you'd like to try out their sparse matrices.
* If you choose to use Numpy (and Scipy), please make sure your Anaconda environment include their latest versions (Numpy 1.12.0; Scipy 0.18.1).
* Test your parsing and PageRank calculations using a handful of tweets, before moving on to the entire file we provide.
* We will evaluate the user ranks you provide as well as the quality of your code. So make sure that your code is clear and readable.

What is the termination condition in your PageRank implementation? Describe it below:

* Since we need only the top 10 ranks, we can use the criterion that we stop when the top 10 don't change from iteration to iteration.

* To ensure that its not something that happens by chance, we can choose a number - let's say 5 and say that the top 10 ranks shouldn't change for atleast 5 iterations.

So that's what the criteria I have used.

In [6]:
# Here add your code to implement a function called PageRanker
# Insert as many cells as you want

def PageRanker(graph, d, numRepeat):
    
    N = len(graph) #graph is the 
    pageRank = {} #dictionary of pageranks for each node and iteration
    pageRank[0] = {} #dictionary of pageranks for each node, Iteration 0
    topTen = {0:[]} #top ten ranks for each iteration, initialized for iteration 0 as null - since all have same PR values
    count = 0
    i = 0
    
    for item in graph:
        pageRank[0][item] = 1/N  #initialized pageranks
        
    while count <= numRepeat: #numRepeat is number of iterations for which top10 should not change
        i += 1
        topTen[i] = []
        pageRank[i] = {}
        
        for item in graph:
            pageRank[i][item] = (1-d)/N #setting the random jumping same for all nodes in every iteration
            
        for item in graph:
            n = len(graph[item])
            if n != 0: #only checking nodes which connect to some other nodes
                for node in graph[item]:
                    pageRank[i][node] += d*(pageRank[i-1][item]/n)
                
        top_10 = sorted(pageRank[i].items(), key=operator.itemgetter(1), reverse=True)[:10] #find top 10
        
        for item in top_10:
            topTen[i].append(item[0])
        
        if topTen[i] == topTen[i-1]: count += 1 #increasing count when 2 successive iterations have same top 10
    
    print count, i #number of iterations where same top 10, total iterations
    
    return pageRank, topTen[i]

In [7]:
# Now let's call your function on the graph you've built. Output the results

start = time.time()
yourPagerank, yourTopranks = PageRanker(yourGraph, 0.9, 4)
end = time.time()
print "Time taken to calculate pagerank is %04.2f seconds"%(end-start)
numIter = len(yourPagerank) - 1

[(yourUsers.keys()[yourUsers.values().index(x)],
  yourPagerank[numIter][x],
  str(yourUsersNames[yourUsers.keys()[yourUsers.values().index(x)]])) for x in yourTopranks]


5 10
Time taken to calculate pagerank is 0.24 seconds


[(158314798, 0.0014407991084827436, 'Real_Liam_Payne'),
 (181561712, 0.0012177851485260277, 'Harry_Styles'),
 (209708391, 0.0011338694117513576, 'onedirection'),
 (72064417, 0.001033536214242238, 'PointlessBlogTv'),
 (105119490, 0.000949046991421453, 'NiallOfficial'),
 (14268057, 0.00087340231284236, 'wittynate'),
 (379961664, 0.0008259281801582455, 'WOWFactsOfLife'),
 (391037985, 0.0007946135118685318, 'twhiddleston'),
 (153074065, 0.0007939878054082239, 'OurCanadianHope'),
 (313525912, 0.0007643829884357868, 'FinnHarries')]

# Part 2: Search Engine Optimization (30 + 5 points)

For this part, your goal is to put on your "[search engine optimization](https://en.wikipedia.org/wiki/Search_engine_optimization)" hat. Your job is to create a webpage that scores highest for the query: **awcv9kjlh scwrlkjf4e** --- two terms, lower case, no quote. As of today (Feb 7, 2017), there are no hits for this query on either Google or Bing. Based on our discussions of search engine ranking algorithms, you know that several factors may impact a page's rank. Your goal is to use this knowledge to promote your own page to the top of the list.

What we're doing here is a form of [SEO contest](https://en.wikipedia.org/wiki/SEO_contest). While you have great latitude in how you approach this problem, you are not allowed to engage in any unethical or illegal behavior. Please read the discussion of "white hat" versus "black hat" SEO over at [Wikipedia](https://en.wikipedia.org/wiki/Search_engine_optimization).


**Rules of the game:**

* Somewhere in the page (possibly in the non-viewable source html) you must include your name or some other way for us to identify you (e.g., your NetID, but not the UIN!).
* Your target page may only be a TAMU student page, a page on your own webserver, a page on a standard blog platform (e.g., wordpress), or some other primarily user-controlled page
* Your target page CAN NOT be a twitter account, a facebook page, a Yahoo Answers or similar page
* No wikipedia vandalism
* No yahoo/wiki answers questions
* No comment spamming of blogs
* If you have concerns/questions/clarifications, please post on Piazza and we will discuss

For your homework turnin for this part, you should provide us the URL of your target page and a brief discussion (2-4 paragraphs) of the strategies you are using. We will issue the query and check the rankings at some undetermined time in the next couple of weeks. You might guess that major search engines take some time to discover and integrate new pages: if I were you, I'd get a target page up immediately.

**Grading:**

* 5 points for providing a valid URL
* 20 points for a well-reasoned discussion of your strategy
* 5 points for your page appearing in the search results by Google or Bing (no matter how is the ranking)

** Bonus: **
* 1 point for your page appearing in the top-20 on Google or Bing
* 1 more point for your page appearing in the top-10 on Google or Bing
* 1 more point for your page appearing in the top-5 on Google or Bing
* 2 more points for your page being ranked first by Google or Bing. And, a vigorous announcement in class, and a high-five for having the top result!

What's the URL of your page?

*ADD YOUR INPUT HERE*

What's your strategy? (2-4 paragraphs)

*ADD YOUR INPUT HERE*